In [1]:
%matplotlib widget
# Load packages and settings
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.min_rows', 50)
import seaborn as sns


import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (12,8)
plt.rcParams["image.cmap"] = "tab10"
plt.rcParams['axes.prop_cycle'] = plt.cycler(color=plt.cm.tab10.colors)
fs_label = 16
parameters = {
                'axes.labelsize': fs_label,
                'axes.titlesize': fs_label+4,
                'xtick.labelsize': fs_label,
                'ytick.labelsize': fs_label, 
                'legend.fontsize': fs_label, 
                'lines.markersize': 10,
                'lines.linewidth': 3
             }
plt.rcParams.update(parameters)
from matplotlib import cm # Colormaps
import matplotlib.colors as colors
# cmap = plt.cm.get_cmap('Dark2',len(ageGroups))

import locale
import matplotlib.dates as mdates
locale.setlocale(locale.LC_TIME,"Danish")
# ax1.xaxis.set_major_formatter(mdates.DateFormatter('%b\n%Y'))
# ax1.spines['top'].set_visible(False) 

import os
# import csv
import math

from datetime import date


saveFigures = True
# saveFigures = False
print('saveFigures is set to: '+str(saveFigures))

print('Done loading packages')

# Define running mean functions
def rnMean(data,meanWidth):
    return np.convolve(data, np.ones(meanWidth)/meanWidth, mode='valid')
def rnTime(t,meanWidth):
    return t[math.floor(meanWidth/2):-math.ceil(meanWidth/2)+1]

saveFigures is set to: True
Done loading packages


In [2]:
# Define paths
rootdir_data = os.getcwd() +"\\..\\DanskeData\\" 

path_data = rootdir_data + "ssi_data\\"
path_dash = rootdir_data + "ssi_dashboard\\"
path_vacc = rootdir_data + "ssi_vacc\\"

path_figs = os.getcwd() +"\\..\\Figures\\" 

In [6]:
# Data is (only) in the file from the most recent tuesday. 
# Should be made smarter, but here hardcoded
tuePath = 'SSI_data_2022-01-04'

dirPath = path_data + tuePath + '\\'

df1 = pd.read_csv(dirPath+'gennembrudsinfektioner_table1.csv',delimiter=';')
df2_C = pd.read_csv(dirPath+'gennembrudsinfektioner_table2_antal_cases.csv',delimiter=';')
df2_H = pd.read_csv(dirPath+'gennembrudsinfektioner_table2_antal_indlagte.csv',delimiter=';')
df2_R = pd.read_csv(dirPath+'gennembrudsinfektioner_table2_antal_repositive.csv',delimiter=';')
df3 = pd.read_csv(dirPath+'gennembrudsinfektioner_table2_incidence_alle.csv',delimiter=';')
df3_C = pd.read_csv(dirPath+'gennembrudsinfektioner_table2_incidence_cases.csv',delimiter=';')
df3_H = pd.read_csv(dirPath+'gennembrudsinfektioner_table2_incidence_indlagte.csv',delimiter=';')

In [7]:
df1

,Ugenummer,antal_personer_alle_Ingen vaccination,antal_personer_alle_Første vaccination,antal_personer_alle_Anden vaccination,antal_personer_alle_Fuld effekt efter primært forløb,antal_personer_alle_Fuld effekt efter revaccination,antal_personer_Ingen vaccination,antal_personer_Første vaccination,antal_personer_Anden vaccination,antal_personer_Fuld effekt efter primært forløb,...,antal_repositive_Ingen vaccination,antal_repositive_Første vaccination,antal_repositive_Anden vaccination,antal_repositive_Fuld effekt efter primært forløb,antal_repositive_Fuld effekt efter revaccination,antal_indlagte_Ingen vaccination,antal_indlagte_Første vaccination,antal_indlagte_Anden vaccination,antal_indlagte_Fuld effekt efter primært forløb,antal_indlagte_Fuld effekt efter revaccination
0,Uge 52-2021,1088643,248331,4537947,3150616,1347647,955611,219496,4229125,2902372,...,2452,233,4373,3656,664,350,42,731,452,273
1,Uge 51-2021,1120351,236206,4519312,3462386,1013194,995529,209132,4229024,3218492,...,1027,122,2395,2154,206,314,27,489,356,124
2,Uge 50-2021,1202341,177247,4497152,3635921,819112,1081390,154383,4225939,3403068,...,747,89,1777,1645,102,283,26,473,370,100
3,Uge 49-2021,1273286,129991,4473880,3817089,625858,1156375,108815,4218930,3591929,...,344,41,431,391,39,241,17,412,349,62
4,Uge 48-2021,1313033,109723,4455244,3943441,490213,1198791,89721,4211339,3723508,...,230,22,125,113,11,222,16,376,320,54
5,Uge 47-2021,1332174,102259,4443969,4025840,398172,1219318,83911,4208455,3810302,...,208,12,100,94,6,219,14,327,301,26
6,Uge 46-2021,1355346,89083,4434226,4092148,325156,1241088,73826,4204821,3878904,...,201,7,84,80,3,210,8,328,296,32
7,Uge 45-2021,1381250,72543,4425201,4164984,244563,1264898,60143,4199767,3952069,...,174,7,61,60,1,145,15,290,268,22
8,Uge 44-2021,1392493,68151,4418617,4241407,160744,1276413,56330,4196104,4028031,...,130,5,39,36,3,160,9,205,191,11
9,Uge 43-2021,1401119,67850,4411169,4277777,115475,1285257,56047,4191234,4065614,...,107,3,39,38,1,113,3,182,167,15
